In [4]:
import google.generativeai as genai
import json
import re
import webbrowser
from urllib.parse import quote

# 모델 생성
model = genai.GenerativeModel('gemini-1.5-flash')

def parse_days(input_str):
    match = re.search(r"\d+", input_str)
    return int(match.group()) if match else 1

def generate_map_urls(place_name):
    """ 장소명을 받아 네이버 지도 및 구글 검색 URL을 생성 """
    encoded_place = quote(place_name)  # URL 인코딩
    naver_map_url = f"https://map.naver.com/v5/search/{encoded_place}"
    google_search_url = f"https://www.google.com/search?q={encoded_place}"
    return naver_map_url, google_search_url

# 사용자 입력 받기
print("여행 일정 추천을 위해 아래 항목들을 입력해주세요:")
location = input("여행지 위치 (예: 제주도, 유럽): ")
days = parse_days(input("여행 기간 (예: 2일, 3): "))
theme = input("여행 테마 (예: 감성 카페 & 맛집): ")

# AI에 메시지 전송
query = f"""
당신은 여행 일정 생성 AI입니다.
사용자의 요청에 맞춰 {location}에서 {days}일 동안 '{theme}' 테마 여행 일정을 생성하세요.

각 날짜마다 정확히 3개의 장소를 추천하세요.

응답은 아래 JSON 형식을 따르세요:
{{
    "itinerary": [
        {{"day": 1, "places": [
            {{"name": "장소1", "type": "{theme}", "location": "장소의 이름 (주소 대신 지도 검색 URL 제공)"}},
            {{"name": "장소2", "type": "{theme}", "location": "장소의 이름 (주소 대신 지도 검색 URL 제공)"}},
            {{"name": "장소3", "type": "{theme}", "location": "장소의 이름 (주소 대신 지도 검색 URL 제공)"}}
        ]}},
        ... (각 날짜별 반복)
    ]
}}
"""

response = model.generate_content(query)

# 응답 처리
try:
    response_data = json.loads(response.text)

    for day in response_data["itinerary"]:
        # 🔥 AI가 응답한 장소 리스트에서 3개만 유지
        day["places"] = day["places"][:3]

        for place in day["places"]:
            place_name = place["name"]
            
            # 🔥 URL을 여기서 추가 (3개 고정 후 처리)
            naver_url, google_url = generate_map_urls(place_name)

            place["naver_map_url"] = naver_url
            place["google_search_url"] = google_url

    # JSON 형태로 결과 출력
    print(json.dumps(response_data, indent=4, ensure_ascii=False))

    # 웹 브라우저에서 네이버 지도 열기 (테스트용)
    if response_data["itinerary"]:
        first_place = response_data["itinerary"][0]["places"][0]
        webbrowser.open(first_place["naver_map_url"])  # 첫 번째 장소의 네이버 지도 열기

except json.JSONDecodeError:
    print("⚠️ 응답이 JSON 형식이 아닙니다. 원본 출력:")
    print(response.text)


여행 일정 추천을 위해 아래 항목들을 입력해주세요:
⚠️ 응답이 JSON 형식이 아닙니다. 원본 출력:
```json
{
  "itinerary": [
    {
      "day": 1,
      "places": [
        {
          "name": "태종대",
          "type": "힐링",
          "location": "https://www.google.com/maps/search/태종대"
        },
        {
          "name": "해운대 해수욕장 (해변 산책)",
          "type": "힐링",
          "location": "https://www.google.com/maps/search/해운대+해수욕장"
        },
        {
          "name": "센텀시티 신세계백화점 스파랜드",
          "type": "힐링",
          "location": "https://www.google.com/maps/search/센텀시티+신세계백화점+스파랜드"
        }
      ]
    },
    {
      "day": 2,
      "places": [
        {
          "name": "감천문화마을 (벽화마을 산책)",
          "type": "힐링",
          "location": "https://www.google.com/maps/search/감천문화마을"
        },
        {
          "name": "송정해수욕장 (카페에서 바다 감상)",
          "type": "힐링",
          "location": "https://www.google.com/maps/search/송정해수욕장"
        },
        {
          "name": "부산 사찰 (해동 용궁사 또는  범어사 선택)",
          "type": "힐